In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

videoFrames = []


def display_frame(frame):

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(rgb_frame)
    plt.axis("off")
    plt.show()
    clear_output(wait=True)  

def main():
    video = "raw_video_feed.mp4"
    capture = cv2.VideoCapture(video)
 
    lower = np.array([50, 66, 43]) 
    upper = np.array([160, 200, 100])  
    

    kernel = np.ones((5, 5), np.uint8)

    while True:
        ret, frame = capture.read()
      
        if not ret:
            break  


        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
    
        mask = cv2.inRange(hsv, lower, upper)
        
        
        cv2.imshow("Mask", mask)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break


        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contours) > 0:
            largest_contour = max(contours, key=cv2.contourArea)
            
          
            M = cv2.moments(largest_contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
            else:
                cx, cy = 0, 0
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray_bgr = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
        blue_part = cv2.bitwise_and(frame, frame, mask=mask)
        mask_inv = cv2.bitwise_not(mask)
        gray_part = cv2.bitwise_and(gray_bgr, gray_bgr, mask=mask_inv)
        final_frame = cv2.add(blue_part, gray_part)
        
        if cx >= 0 and cy >= 0:

            cv2.circle(final_frame, (cx, cy), 20, (0, 0, 255), -1)

        
        videoFrames.append(final_frame)
        display_frame(final_frame)

    capture.release()

main()

frame_height, frame_width = videoFrames[0].shape[:2]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
out = cv2.VideoWriter('output_video.mp4', fourcc, 30, (frame_width, frame_height))

for frame in videoFrames:
    out.write(frame)

out.release()
print("Video saved as 'output_video.mp4'")



KeyboardInterrupt: 